### Dealing with categorical variables

One hot encode
- The categorical value represents the numerical value of the entry in the dataset.
- curse of dimensionality makes dimensionality increase exponential
- lose the explicit one columns relationship of the feature

Label encode
- `0, 1, 2, 3`
- is an ordinal encoding - even if feature is not ordinal

Mean encode
- put the training data average for the target for that class
- could also use other statistics like median, quantiles or variance

Target encode
- features are replaced with a blend of the posterior probability of the target for the given particular categorical value and the prior probability of the target over all the training data.
- the posterior probability is the probability p(t = 1 | x = ci) where t denotes the target, x is the input and ci is the i-th category given the input was the category ci.
- they are not generated for the test data. 
- We usually save the target encodings obtained from the training data set and use the same encodings to encode features in the test data set.

BaseN Encoding
- In binary encoding, we convert the integers into binary i.e base 2.
---> First, the categories are encoded as ordinal, then those integers are converted into binary code, 
then the digits from that binary string are split into separate columns. (this concept uses Hashing!) Read More here:https://ieeexplore.ieee.org/document/5474379
- A base of 1 is equivalent to one-hot encoding, a base of 2 is equivalent to binary encoding.
- BaseN allows us to convert the integers with any value of the base.
- ideal for columns with large categorical types

In [2]:
import category_encoders as ce
import pandas as pd
import numpy as np

#import warnings
#warnings.filterwarnings("ignore")


data = pd.read_csv('./data/cars/cars.csv',index_col=0)
data.head()

,Foreign/Local Used,color,wheel drive,Automation,seat-make,price,description,make-year,manufacturer
0,Foreign Used,Black,4,Automatic,Leather,17500000,2014 Lexus LX,2014,Lexus
1,Foreign Used,Black,4,Automatic,Leather,13000000,2012 Toyota Sequoia,2012,Toyota
2,Foreign Used,Blue,4,Automatic,Cloth,6500000,2007 Toyota FJ CRUISER,2007,Toyota
3,Foreign Used,Black,4,Automatic,Leather,4700000,2005 Lexus GX,2005,Lexus
4,Foreign Used,Grey,4,Automatic,Leather,3800000,2005 Toyota 4-Runner,2008,Toyota


In [2]:
data.color.value_counts()

Black         390
Silver        243
Grey          131
Red            84
White          81
Blue           76
Gold           57
Maroon         47
Dark Grey      32
Dark Blue      25
Dark Green     13
Green           8
Other           3
Name: color, dtype: int64

In [3]:
#Label Encoding the color column
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()  #instantiate the Label Encoder
data['color'] = le.fit_transform(data['color'])

#it's ideal to always instantiate new LabelEncoders for different columns

In [4]:
data.head()

,Foreign/Local Used,color,wheel drive,Automation,seat-make,price,description,make-year,manufacturer
0,Foreign Used,0,4,Automatic,Leather,17500000,2014 Lexus LX,2014,Lexus
1,Foreign Used,0,4,Automatic,Leather,13000000,2012 Toyota Sequoia,2012,Toyota
2,Foreign Used,1,4,Automatic,Cloth,6500000,2007 Toyota FJ CRUISER,2007,Toyota
3,Foreign Used,0,4,Automatic,Leather,4700000,2005 Lexus GX,2005,Lexus
4,Foreign Used,7,4,Automatic,Leather,3800000,2005 Toyota 4-Runner,2008,Toyota


In [5]:
data['seat-make'].value_counts()

Leather    920
Cloth      270
Name: seat-make, dtype: int64

In [6]:
data.dtypes

Foreign/Local Used    object
color                  int64
wheel drive            int64
Automation            object
seat-make             object
price                  int64
description           object
make-year              int64
manufacturer          object
dtype: object

In [7]:
#one hot encoding for Foreign/Local Used column
import category_encoders as ce

# create an object of the OneHotEncoder
ce_one = ce.OneHotEncoder(cols=['Foreign/Local Used','Automation']) 

ce_one.fit_transform(data)

,Foreign/Local Used_1,Foreign/Local Used_2,color,wheel drive,Automation_1,Automation_2,seat-make,price,description,make-year,manufacturer
0,1,0,0,4,1,0,Leather,17500000,2014 Lexus LX,2014,Lexus
1,1,0,0,4,1,0,Leather,13000000,2012 Toyota Sequoia,2012,Toyota
2,1,0,1,4,1,0,Cloth,6500000,2007 Toyota FJ CRUISER,2007,Toyota
3,1,0,0,4,1,0,Leather,4700000,2005 Lexus GX,2005,Lexus
4,1,0,7,4,1,0,Leather,3800000,2005 Toyota 4-Runner,2008,Toyota
...,...,...,...,...,...,...,...,...,...,...,...
286,1,0,4,4,1,0,Leather,10000000,2012 LR4,2012,LR4
287,1,0,0,4,1,0,Leather,3500000,2012 Toyota Camry,2012,Toyota
288,1,0,10,4,1,0,Leather,6500000,2010 Lexus RX,2010,Lexus
289,1,0,8,4,1,0,Leather,8700000,2013 Toyota Highlander,2013,Toyota


In [8]:
#get_dummies
pd.get_dummies(data['Foreign/Local Used']).head(30)

#Convert categorical variable into dummy/indicator variables.

,Foreign Used,Locally Used
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


In [13]:
data

,Foreign/Local Used,color,wheel drive,Automation,seat-make,price,description,make-year,manufacturer
0,Foreign Used,Black,4,Automatic,Leather,17500000,2014 Lexus LX,2014,Lexus
1,Foreign Used,Black,4,Automatic,Leather,13000000,2012 Toyota Sequoia,2012,Toyota
2,Foreign Used,Blue,4,Automatic,Cloth,6500000,2007 Toyota FJ CRUISER,2007,Toyota
3,Foreign Used,Black,4,Automatic,Leather,4700000,2005 Lexus GX,2005,Lexus
4,Foreign Used,Grey,4,Automatic,Leather,3800000,2005 Toyota 4-Runner,2008,Toyota
...,...,...,...,...,...,...,...,...,...
286,Foreign Used,Dark Grey,4,Automatic,Leather,10000000,2012 LR4,2012,LR4
287,Foreign Used,Black,4,Automatic,Leather,3500000,2012 Toyota Camry,2012,Toyota
288,Foreign Used,Red,4,Automatic,Leather,6500000,2010 Lexus RX,2010,Lexus
289,Foreign Used,Maroon,4,Automatic,Leather,8700000,2013 Toyota Highlander,2013,Toyota


In [9]:
#Target encoding
ce_te = ce.TargetEncoder(cols=['seat-make'])

#column to perform encoding
X = data['seat-make']
y = data['color']

#create an object of the Targetencoder
ce_te.fit(X,y)

ce_te.transform(X).head()

,seat-make
0,5.253261
1,5.253261
2,5.918519
3,5.253261
4,5.253261


In [3]:
# make some data
example_df = pd.DataFrame({
 'class' : ['a', 'b', 'a', 'b', 'd', 'e', 'd', 'f', 'g', 'h', 'h', 'k', 'h', 'i', 's', 'p', 'z']})
example_df

,class
0,a
1,b
2,a
3,b
4,d
5,e
6,d
7,f
8,g
9,h


In [6]:
# create an object of the BaseNEncoder
ce_baseN4 = ce.BaseNEncoder(cols=['class'],base=2)
# fit and transform and you will get the encoded data
ce_baseN4.fit_transform(example_df).head(12)

,class_0,class_1,class_2,class_3
0,0,0,0,1
1,0,0,1,0
2,0,0,0,1
3,0,0,1,0
4,0,0,1,1
5,0,1,0,0
6,0,0,1,1
7,0,1,0,1
8,0,1,1,0
9,0,1,1,1


In [2]:
#mean encode
def mean_encode(data, col, on):
    group = data.groupby(col).mean()
    mapper = {k: v for k, v in zip(group.index, group.loc[:, on].values)}

    data.loc[:, col] = data.loc[:, col].replace(mapper)
    data.loc[:, col].fillna(value=np.mean(data.loc[:, col]), inplace=True)

    return data

In [3]:
#example dataframe_1
store1 = pd.DataFrame({'store': ['Alpha'] * 3,
         'Sales': [1000, 2000, 3000],
         'noise': [0, 0, 0]})

#example dataframe_2
store2 = pd.DataFrame(
        {'store': ['Beta'] * 3,
         'Sales': [100, 200, 300],
         'noise': [0, 0, 0]})

data = pd.concat([store1, store2], axis=0)  #concat dataframe
#np.testing.assert_array_equal(data.loc[:, 'store'],np.array([200, 200, 200, 20, 20, 20]))

In [4]:
data

,store,Sales,noise
0,Alpha,1000,0
1,Alpha,2000,0
2,Alpha,3000,0
0,Beta,100,0
1,Beta,200,0
2,Beta,300,0


In [5]:
mean_encode(data, col='store', on='Sales')

,store,Sales,noise
0,2000.0,1000,0
1,2000.0,2000,0
2,2000.0,3000,0
0,200.0,100,0
1,200.0,200,0
2,200.0,300,0


In [16]:
def median_encoder(df, col, on):
    group = df.groupby(col).median()
    mapper = {a: b for a, b in zip(group.index, group.loc[:, on].values)}

    df.loc[:, col] = df.loc[:, col].replace(mapper)
    df.loc[:, col].fillna(value=np.median(df.loc[:, col]), inplace=True)

    return df